In [1]:
import pandas as pd
import numpy as np

# Read the data
shows = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/complete_manually_cleaned_data.csv')
actor_rankings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/actor_rankings.csv')
writer_rankings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/writer_rankings.csv')
director_rankings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/director_rankings.csv')

In [2]:
principals = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.principals.tsv', sep='\t')

In [3]:
crew = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.crew.tsv', sep='\t')

In [4]:
# delete prinicpals with cateegoy not actor, director, or writer
principals2 = principals[principals['category'].isin(['actor','actress', 'self'])]

In [5]:
# isolate tconst column of shows and merge with principals
shows_tconst = pd.DataFrame(shows['tconst'])
shows_tconst_director = shows_tconst.merge(crew, how='left', left_on='tconst', right_on='tconst')
shows_tconst_writer = shows_tconst.merge(crew, how='left', left_on='tconst', right_on='tconst')
shows_tconst_actor = shows_tconst.merge(principals2[principals2['category'].isin(['actor', 'actress', 'self'])][['tconst', 'nconst']], on='tconst', how='left')

In [6]:
shows_tconst_director['directors'] = shows_tconst_director['directors'].str.split(',')
shows_tconst_director = shows_tconst_director[['tconst', 'directors']].explode('directors')

In [7]:
shows_tconst_writer['writers'] = shows_tconst_writer['writers'].str.split(',')
shows_tconst_writer = shows_tconst_writer[['tconst', 'writers']].explode('writers')

In [8]:
# add rankings to each tconst/nconst pair
shows_tconst_director = shows_tconst_director.merge(director_rankings, left_on='directors', right_on='director', how='left')
shows_tconst_writer = shows_tconst_writer.merge(writer_rankings, left_on='writers', right_on='writer', how='left')
shows_tconst_actor = shows_tconst_actor.merge(actor_rankings, left_on='nconst', right_on='actor', how='left')

In [9]:
# take only top 3 actors per show
shows_tconst_actor = shows_tconst_actor.sort_values(by=['tconst', 'total_votes'], ascending=True).groupby('tconst').head(3)
# take only top 2 writers per show
shows_tconst_writer = shows_tconst_writer.sort_values(by=['tconst', 'total_votes'], ascending=True).groupby('tconst').head(3)
# take only top 1 directors per show
shows_tconst_director = shows_tconst_director.sort_values(by=['tconst', 'total_votes'], ascending=True).groupby('tconst').head(1)

In [10]:
# remove all columnds but tconst and rank
shows_tconst_director = shows_tconst_director[['tconst', 'total_votes']]
shows_tconst_writer = shows_tconst_writer[['tconst', 'total_votes']]
shows_tconst_actor = shows_tconst_actor[['tconst', 'total_votes']]

In [11]:
# group by tconst and average rank of all directors/writers/actors
shows_tconst_director = shows_tconst_director.groupby('tconst').mean().reset_index()
shows_tconst_writer = shows_tconst_writer.groupby('tconst').mean().reset_index()
shows_tconst_actor = shows_tconst_actor.groupby('tconst').mean().reset_index()

In [12]:
# merge with shows
shows = shows.merge(shows_tconst_director, on='tconst', how='left')
# rename rank column
shows = shows.rename(columns={'total_votes': 'director_votes'})
shows = shows.merge(shows_tconst_writer, on='tconst', how='left')
# rename rank column
shows = shows.rename(columns={'total_votes': 'writer_votes'})
shows = shows.merge(shows_tconst_actor, on='tconst', how='left')
# rename rank column
shows = shows.rename(columns={'total_votes': 'actor_votes'})

In [13]:
# sort shows by actor_rank
shows = shows.sort_values(by='actor_votes', ascending=True)

In [14]:
shows

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,averageRating,numVotes,titleType,runtimeMinutes,genres,director_votes,writer_votes,actor_votes
620,First Class: Season 1,2022,20220623,True,first class,1.0,True,tt20597418,2.6,131.0,tvSeries,45,Reality-TV,NaN,NaN,6.0
1902,The Puppet Master: Hunting the Ultimate Conman...,2022,20220118,True,the puppet master hunting the ultimate conman,1.0,True,tt15479704,7.0,7045.0,tvMiniSeries,127,"Crime,Documentary",1048.0,NaN,6.0
1622,"Strip Down, Rise Up",2021,20210205,False,strip down rise up,NaN,False,tt13787118,5.3,680.0,movie,112,Documentary,1973.0,1099.0,8.0
939,Karma's World: Season 3,2022,20220707,True,karmas world,3.0,True,tt13321232,6.6,178.0,tvSeries,11,"Animation,Comedy,Family",870.0,312.0,10.0
121,Amina,2021,20211104,True,amina,NaN,False,tt15847872,4.0,338.0,movie,105,"Action,Adventure,Biography",276.0,48.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,WWII in Color: Road to Victory: Season 1,2021,20211222,True,wwii in color road to victory,1.0,True,tt16477402,8.2,1845.0,tvSeries,50,"Documentary,History,War",310.0,NaN,NaN
2137,We Are: The Brooklyn Saints: Season 1,2021,20210129,False,we are the brooklyn saints,1.0,True,tt13656220,6.5,164.0,tvSeries,\N,"Documentary,Sport",6846.0,NaN,NaN
2154,White Hot: The Rise & Fall of Abercrombie & Fitch,2022,20220419,True,white hot the rise fall of abercrombie fitch,NaN,False,tt19034522,5.7,4483.0,movie,88,Documentary,11631.0,NaN,NaN
2164,Why Did You Kill Me?,2021,20210414,False,why did you kill me,NaN,False,tt14318430,5.6,4292.0,movie,83,"Crime,Documentary",NaN,NaN,NaN


In [15]:
# isolate movies    
movies = shows[shows['titleType'] == 'movie']

In [16]:
movies

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,averageRating,numVotes,titleType,runtimeMinutes,genres,director_votes,writer_votes,actor_votes
1622,"Strip Down, Rise Up",2021,20210205,False,strip down rise up,NaN,False,tt13787118,5.3,680.0,movie,112,Documentary,1973.0,1099.0,8.0
121,Amina,2021,20211104,True,amina,NaN,False,tt15847872,4.0,338.0,movie,105,"Action,Adventure,Biography",276.0,48.0,10.0
1007,Lokillo: Nothing's the Same,2021,20210812,False,lokillo nothings the same,NaN,False,tt14494898,4.8,55.0,movie,63,Comedy,NaN,NaN,17.0
1377,Ratones Paranoicos: The Band that Rocked Argen...,2021,20210106,False,ratones paranoicos the band that rocked argentina,NaN,False,tt13794154,7.1,128.0,movie,\N,"Biography,Documentary,Music",NaN,NaN,18.0
63,Adam by Eve: A live in Animation,2022,20220315,True,adam by eve a live in animation,NaN,False,tt18274178,6.1,425.0,movie,58,"Animation,Music",NaN,NaN,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,Trust No One: The Hunt for the Crypto King,2022,20220330,True,trust no one the hunt for the crypto king,NaN,False,tt15479902,6.3,4868.0,movie,90,"Crime,Documentary",919.0,NaN,NaN
2097,Untold: Crime & Penalties,2021,20210831,False,untold crime penalties,NaN,False,tt15101956,7.4,3838.0,movie,85,"Crime,Documentary,Sport",35706.0,NaN,NaN
2098,Untold: Deal With the Devil,2021,20210817,False,untold deal with the devil,NaN,False,tt15085794,7.3,2422.0,movie,77,"Crime,Documentary,Sport",2677.0,NaN,NaN
2154,White Hot: The Rise & Fall of Abercrombie & Fitch,2022,20220419,True,white hot the rise fall of abercrombie fitch,NaN,False,tt19034522,5.7,4483.0,movie,88,Documentary,11631.0,NaN,NaN


In [17]:
print('director_count: ', movies['director_votes'].isna().sum())
print('writer_count: ', movies['writer_votes'].isna().sum())
print('actor_count: ', movies['actor_votes'].isna().sum())
# total number of movies
print('total: ', len(movies))

director_count:  47
writer_count:  111
actor_count:  28
total:  741


director_count:  89
writer_count:  70
actor_count:  102
total:  2226

In [18]:
# write shows to csv
shows.to_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/showsWithPrinicpalVotes.csv', index=False)